In [38]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium

In [23]:
spark = SparkSession.builder.master('local[*]') \
    .appName('Métricas') \
    .getOrCreate()

<p>1 – Quantidade de ônibus em operação</p>

In [24]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_agrupada/"
file_list = os.listdir(directory_path)
dataframes_agrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_agrupados.append(df)



In [25]:
def union_all(dfs):
    return reduce(DataFrame.unionAll, dfs)

In [26]:
dataframes_agrupados_completo = union_all(dataframes_agrupados)
dataframes_agrupados_completo.show(truncate=False)

+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|LINHA  |CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM  |LETREIRO_DESTINO     |QTDE_VEICULOS_OPERACAO|DATA_EXTRACAO_API|
+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|3054-10|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |08:50   |2023-09-15   |3054-10          |33743               |HOSP. SAPOPEMBA  |JD. PALANQUE         |6                     |2023-09

In [27]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)
 |-- DATA_EXTRACAO_API: date (nullable = true)



In [28]:
dataframes_agrupados_completo = dataframes_agrupados_completo.drop('DATA_EXTRACAO_API')
dataframes_agrupados_completo.show()

+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|  LETREIRO_ORIGEM|    LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|3054-10|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   08:50|   2023-09-15|          3054-10|               33743|  HOSP. SAPOPEMBA|        JD. PALANQUE|                     6|
|6026-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   08:50|   2023-09-15|          6026-10|                  87| TERM. STO. AMARO|          JD. ICARAÍ|                     5|
|857A-10|          7|      CONSÓRCIO KBPX|KBPX ADMINISTRAÇÃ.

In [29]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)



In [30]:
@f.udf(returnType=t.StringType())
def turno(hora: str):
    hora_formatada = datetime.strptime(hora, '%H:%M').time()
    if 0 <= hora_formatada.hour < 6:
        return 'Madrugada'
    elif 6 <= hora_formatada.hour < 12:
        return 'Manhã'
    elif 12 <= hora_formatada.hour < 18:
        return 'Tarde'
    else:
        return 'Noite'
        

In [31]:
turno('23:55')

Column<'turno(23:55)'>

In [32]:
dataframes_agrupados_completo = dataframes_agrupados_completo.withColumn('TURNO', turno(f.col('HORA_API'))) 

df_filter = dataframes_agrupados_completo.select(
    dataframes_agrupados_completo.CONSOCIO,
    dataframes_agrupados_completo.TURNO,
    dataframes_agrupados_completo.DATA_EXTRACAO ,
    dataframes_agrupados_completo.LETREIRO_ORIGEM,
    dataframes_agrupados_completo.LETREIRO_DESTINO,
    dataframes_agrupados_completo.HORA_API,
    dataframes_agrupados_completo.LETREIRO_COMPLETO,
    dataframes_agrupados_completo.QTDE_VEICULOS_OPERACAO,
    dataframes_agrupados_completo.CODIGO_IDENTIFICADOR,
    ).filter(
        (dataframes_agrupados_completo.DATA_EXTRACAO == '2023-09-15' ) & 
        (dataframes_agrupados_completo.TURNO == 'Noite') &
        (dataframes_agrupados_completo.LETREIRO_COMPLETO == '9047-10')
        ) \
        
df_filter.show(truncate=False)

+-----------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CONSOCIO               |TURNO|DATA_EXTRACAO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|LETREIRO_COMPLETO|QTDE_VEICULOS_OPERACAO|CODIGO_IDENTIFICADOR|
+-----------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:00   |9047-10          |9                     |33651               |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:00   |9047-10          |5                     |883                 |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |7                     |33651               |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:05   |9047-10      

In [33]:
df_filter.groupBy(['CONSOCIO', 'LETREIRO_COMPLETO',  'LETREIRO_ORIGEM', 'LETREIRO_DESTINO','HORA_API', 'TURNO']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULOS_OPERACAO')
    ) \
    .orderBy('HORA_API')\
    .show(truncate=False)

+-----------------------+-----------------+---------------+----------------+--------+-----+----------------------------+
|CONSOCIO               |LETREIRO_COMPLETO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|TURNO|QUANTIDADE_VEICULOS_OPERACAO|
+-----------------------+-----------------+---------------+----------------+--------+-----+----------------------------+
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:00   |Noite|14                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:05   |Noite|13                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:10   |Noite|15                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:15   |Noite|14                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:20   |Noite|15                          |
|CONSÓRCIO TRANSNOROESTE|9047-10

In [34]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_desagrupada"
file_list = os.listdir(directory_path)
dataframes_desagrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_desagrupados.append(df)



In [35]:
dataframes_desagrupados_completo = union_all(dataframes_desagrupados)
dataframes_desagrupados_completo = dataframes_desagrupados_completo.drop('LINHA')
dataframes_desagrupados_completo = dataframes_desagrupados_completo.withColumn('TURNO', turno(f.col('HORA_API')))
dataframes_desagrupados_completo.show(truncate=False)

+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |DATA_EXTRACAO_API|TURNO|
+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |2023-09-15 

In [74]:
dataframe_prefixo_onibus = dataframes_desagrupados_completo \
    .select(
        dataframes_desagrupados_completo.HORA_API,
        dataframes_desagrupados_completo.PREFIXO_ONIBUS,
        dataframes_desagrupados_completo.LETREIRO_COMPLETO,
        dataframes_desagrupados_completo.LATITUDE,
        dataframes_desagrupados_completo.LONGITUDE,
    ) \
    .filter(
        (dataframes_desagrupados_completo.PREFIXO_ONIBUS == '66821') &
        (dataframes_desagrupados_completo.TURNO == 'Noite') &
        (dataframes_desagrupados_completo.DATA_EXTRACAO == '2023-09-18')
    )



dataframe_prefixo_onibus.show(33)

+--------+--------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+-----------------+-------------------+-------------------+
|   18:00|         66821|          6026-10|-23.685382250000004|       -46.69115925|
|   18:05|         66821|          6026-10|        -23.7037105|         -46.686306|
|   18:10|         66821|          6026-10|-23.703379499999997|         -46.688385|
|   18:15|         66821|          6026-10|-23.708141750000003|        -46.7000405|
|   18:20|         66821|          6026-10|-23.717132999999997| -46.69933425000001|
|   18:25|         66821|          6026-10|        -23.7227975|        -46.7083715|
|   18:32|         66821|          6026-10|        -23.7329835|        -46.7143215|
|   18:35|         66821|          6026-10|       -23.73797275|        -46.7098225|
|   18:40|         66821|          6026-10|        -23.7414215|        -46.7

In [75]:
def haversine(lat1, lon1, lat2, lon2):

    r = 6371.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

 
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = r * c

    return distance

@pandas_udf(DoubleType())
def haversine_udf(lat, lon):
    distances = []
    for i in range(len(lat)):
        
        if i == 0:
            distances.append(0.0)  
        else:
            distance = haversine(lat[i], lon[i], lat[i - 1], lon[i - 1])
            distances.append(distance)
    return pd.Series(distances)

df= dataframe_prefixo_onibus.withColumn("DISTANCIA", haversine_udf(dataframe_prefixo_onibus["LATITUDE"], dataframe_prefixo_onibus["LONGITUDE"]))


df.show(33)


soma_distancias = df.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')



+--------+--------------+-----------------+-------------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|          DISTANCIA|
+--------+--------------+-----------------+-------------------+-------------------+-------------------+
|   18:00|         66821|          6026-10|-23.685382250000004|       -46.69115925|                0.0|
|   18:05|         66821|          6026-10|        -23.7037105|         -46.686306|  2.097063785225343|
|   18:10|         66821|          6026-10|-23.703379499999997|         -46.688385|0.21484791182168067|
|   18:15|         66821|          6026-10|-23.708141750000003|        -46.7000405|  1.299465514754715|
|   18:20|         66821|          6026-10|-23.717132999999997| -46.69933425000001|  1.002363514667756|
|   18:25|         66821|          6026-10|        -23.7227975|        -46.7083715|  1.114960891347674|
|   18:32|         66821|          6026-10|        -23.7329835| 

Soma das distâncias: 31.62 km


In [76]:
lista_de_pontos = dataframe_prefixo_onibus.rdd.flatMap(lambda linha : [[linha.LATITUDE,linha.LONGITUDE] ]).collect()
lista_de_pontos[int(len(lista_de_pontos) // 2)] 

[-23.737052499999997, -46.710246]

In [77]:
dataframe_prefixo_onibus.select(
    dataframe_prefixo_onibus.HORA_API
).distinct().show()

+--------+
|HORA_API|
+--------+
|   21:02|
|   18:05|
|   18:32|
|   19:15|
|   20:50|
|   19:10|
|   19:35|
|   20:30|
|   20:00|
|   18:40|
|   21:05|
|   18:15|
|   20:25|
|   19:05|
|   18:35|
|   20:45|
|   19:20|
|   20:15|
|   20:40|
|   20:20|
+--------+
only showing top 20 rows



In [78]:
m = folium.Map(
    location=lista_de_pontos[int(len(lista_de_pontos) // 2)] ,
    zoom_start=10,
    control_scale=True)
for ponto in lista_de_pontos:
   folium.Marker(
    location=ponto,
    popup='Teste',
    icon=folium.Icon(
        color='red', 
        icon='icon-sigin'
    )
).add_to(m)
m
